**Trần Duy Tân - 22550020**

In [9]:
pip install findspark

In [10]:
pip install pyspark

**Câu 1: Xuất số lượng phim được làm mỗi năm. Kết quả được sắp xếp theo năm và có định dạng sau đây. (1 điểm)**

**Load Data**

In [16]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
spark = SparkSession.builder.appName("DataFrame").getOrCreate()
from pyspark.sql.functions import avg, explode, split
# Read movie and rating data
movieDF = spark.read.options(delimiter=',').schema('movie_id INT, title STRING, genres STRING').csv("movies_small.csv")
ratingDF = spark.read.options(delimiter=',').schema('user_id INT, movie_id INT, rating DOUBLE, timestamp STRING').csv("ratings_small.csv")


**Câu 1 Xuất số lượng phim được làm mỗi năm. Kết quả được sắp xếp theo năm và có định dạng sau đây. (1 điểm)**

In [14]:
temp1 = movieDF.withColumn("Year",
                           f.when(f.regexp_extract("title", r"\((\d{4}(\–\d{4})?)\)", 1) != "",
                                  f.regexp_extract("title", r"\((\d{4}(\–\d{4})?)\)", 1).cast("int"))
                            .otherwise(f.lit(' '))
                          )
temp1 = temp1.na.drop(subset=["Year"])
result1 = temp1.groupBy("Year").agg(f.count("year").alias("Num_of_movies"))
result1 = result1.sort(result1['Year'].cast('int'))
result1.show(20, truncate=False)


+----+-------------+
|Year|Num_of_movies|
+----+-------------+
|    |13           |
|1902|1            |
|1903|1            |
|1908|1            |
|1915|1            |
|1916|4            |
|1917|1            |
|1919|1            |
|1920|2            |
|1921|1            |
|1922|1            |
|1923|4            |
|1924|5            |
|1925|4            |
|1926|5            |
|1927|7            |
|1928|4            |
|1929|4            |
|1930|5            |
|1931|14           |
+----+-------------+
only showing top 20 rows



 **câu 2 : Xuất số lượng phim thuộc về mỗi dòng phim được làm mỗi năm. Kết quả được sắp xếp theo năm. Số phim của mỗi
dòng được trình bày trong từng cột như sau: ( 2 điểm)**

In [18]:
cau2 = movieDF.withColumn("Year", f.regexp_extract("title", r"\((\d{4})\)", 1))
cau2 = cau2.withColumn("Genre", f.explode(f.split("genres", "\|")))
result_cau2 = cau2.groupBy("Year").pivot("Genre").count()
result_cau2= result_cau2.orderBy("Year")
result_cau2.show(10, truncate=False)

+----+------------------+------+---------+---------+--------+------+-----+-----------+-----+-------+---------+------+----+-------+-------+-------+------+--------+----+-------+------+
|Year|(no genres listed)|Action|Adventure|Animation|Children|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|IMAX|Musical|Mystery|Romance|Sci-Fi|Thriller|War |Western|genres|
+----+------------------+------+---------+---------+--------+------+-----+-----------+-----+-------+---------+------+----+-------+-------+-------+------+--------+----+-------+------+
|    |9                 |1     |NULL     |NULL     |NULL    |NULL  |NULL |NULL       |2    |NULL   |NULL     |NULL  |NULL|NULL   |NULL   |NULL   |2     |2       |NULL|NULL   |1     |
|1902|NULL              |1     |1        |NULL     |NULL    |NULL  |NULL |NULL       |NULL |1      |NULL     |NULL  |NULL|NULL   |NULL   |NULL   |1     |NULL    |NULL|NULL   |NULL  |
|1903|NULL              |NULL  |NULL     |NULL     |NULL    |NULL  |1    |NULL       

**Câu 3 Với mỗi userID, xuất điểm đánh giá trung bình của user đó cho mỗi dòng phim. Kết quả được sắp xếp theo userID.
Kết quả của từng dòng phim được trình bày trong mỗi cột như sau: (2 điểm)**

In [19]:
# Join và tính trung bình rating theo genres và userId
joinCau3 = ratingDF.join(movieDF, on=['movie_id'], how="inner")
cau3 = joinCau3.withColumn("Genre", explode(split("genres", "\|")))
result_cau3 = cau3.groupBy("user_id", "Genre").agg(avg("rating").alias("avg_rating"))
result_cau3 = result_cau3.groupBy("user_id").pivot("Genre").avg("avg_rating")
# Sắp xếp theo userId và hiển thị 100 dòng mà không truncate
sorted_cau3 = result_cau3.orderBy("user_id")
sorted_cau3.show(10, truncate=False)


+-------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+---------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|user_id|(no genres listed)|Action            |Adventure         |Animation         |Children          |Comedy            |Crime             |Documentary      |Drama             |Fantasy           |Film-Noir|Horror            |IMAX              |Musical           |Mystery           |Romance           |Sci-Fi            |Thriller          |War               |Western           |
+-------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+---------+------------------+-------------

**Câu 4 :Với mỗi phim, xuất ra tên, năm, số lượng ratings, và điểm rating trung bình (từ mọi users) cho bộ phim đó. Kết quả
được sắp xếp theo năm, sau đó theo tên phim như sau: (2 điểm)**

In [21]:
cau4 = movieDF.join(ratingDF, on='movie_id', how='inner')
resultDF = cau4.groupBy('title', 'genres', 'movie_id').agg(
    f.count('rating').alias('Num_rating'),
    f.avg('rating').alias('Average_rating')
)
resultDF = resultDF.withColumn('Year', f.regexp_extract('title', r'\((\d{4})\)', 1))
resultDF = resultDF.select('Year', 'title', 'Num_rating', 'Average_rating').withColumnRenamed('title', 'Movie_name') \
    .orderBy(['Year', 'title'])
resultDF.show(10, truncate=False)

+----+---------------------------------+----------+--------------+
|Year|Movie_name                       |Num_rating|Average_rating|
+----+---------------------------------+----------+--------------+
|    |Babylon 5                        |2         |2.25          |
|    |Black Mirror                     |1         |5.0           |
|    |Cosmos                           |2         |4.5           |
|    |Death Note: Desu nôto (2006–2007)|1         |5.0           |
|    |Generation Iron 2                |1         |3.5           |
|    |Hyena Road                       |1         |2.0           |
|    |Maria Bamford: Old Baby          |1         |1.0           |
|    |Moonlight                        |1         |5.0           |
|    |Nocturnal Animals                |1         |3.0           |
|    |Paterson                         |1         |4.5           |
+----+---------------------------------+----------+--------------+
only showing top 10 rows

